### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report,accuracy_score,\
log_loss,confusion_matrix
import math
import warnings
warnings.filterwarnings('ignore')

### Reading Data

In [2]:
df=pd.read_csv("bank.xls")

In [3]:
df.drop(['ID'],axis=1,inplace=True)

### Downsampling Data

In [4]:
one_class=df[df['Personal Loan']==1]
zero_class=df[df['Personal Loan']==0]

In [5]:
zero_downsample = resample(zero_class,
             replace=True,
             n_samples=len(one_class),
             random_state=42)

In [6]:
data=pd.concat([zero_downsample,one_class])

In [7]:
X=data.drop(['Personal Loan'],axis=1).values
Y=data['Personal Loan'].values

### Splitting Data

In [8]:
X_train,X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [9]:
X_train.shape

(768, 12)

In [10]:
sc=StandardScaler()

In [11]:
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)

### Ant Colony Optimisation Algorithm for Weight Optimisation

In [12]:
import numpy as np
from sklearn.metrics import accuracy_score, log_loss

class AntColonyOptimizer:
    def __init__(self, X_train, y_train, num_ants=30, num_generations=30, hidden_layer=10):
        self.X_train = X_train
        self.y_train = y_train
        self.num_ants = num_ants
        self.num_generations = num_generations
        self.hidden_layer = hidden_layer
        self.input_layer = X_train.shape[1]
        self.output_layer = 1
        self.weights = (self.input_layer+1)*self.hidden_layer + (self.hidden_layer+1)*self.output_layer
        self.weight_points = np.random.rand(self.weights, self.hidden_layer)
        self.initial_ph = 1 / (self.input_layer + self.hidden_layer + self.output_layer)
        self.pheromones = np.ones((self.weights, self.hidden_layer)) * self.initial_ph
        self.inputs = X_train

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def prob_matrix(self, pheromone):
        p = []
        for i in range(len(pheromone)):
            p.append(pheromone[i] / np.sum(pheromone))
        return p

    def generate_ant(self):
        ant = []
        temp_memory = []
        for i in range(len(self.pheromones)):
            num = 0
            prob = self.prob_matrix(self.pheromones[i])
            if all(p == prob[0] for p in prob):
                num = np.random.randint(low=0, high=len(prob))

            else:
                num = np.where(prob == np.max(prob))[0][0]
            ant.append(self.weight_points[i][num])
            temp_memory.append(num)
        return ant, temp_memory

    def update_pheromones(self, ants):
        pheromones_ = np.zeros_like(self.pheromones)
        for ant in ants:
            pheromones_ += ant['pheromones']
        return pheromones_

    def traverse(self, ant):
        for i in range(len(self.pheromones)):
            num = 0
            prob = self.prob_matrix(self.pheromones[i])
            num = np.where(prob == np.max(prob))[0][0]
            ant['weights'][i] = self.weight_points[i][num]
        return ant

    def forward(self, inputs, weights):
        weight1 = np.reshape(weights[:self.hidden_layer*self.input_layer], (self.hidden_layer, self.input_layer))
        bias1 = np.reshape(weights[self.hidden_layer*self.input_layer:self.hidden_layer*self.input_layer+self.hidden_layer], (self.output_layer, self.hidden_layer))
        weight2 = np.reshape(weights[self.hidden_layer*self.input_layer+self.hidden_layer:-1], (self.output_layer, self.hidden_layer))
        bias2 = np.reshape(weights[-1], (self.output_layer, self.output_layer))

        A0 = inputs
        Z1 = inputs.dot(weight1.T) + bias1
        A1 = Z1
        Z2 = A1.dot(weight2.T) + bias2
        A2 = self.sigmoid(Z2)
        y_pred = np.round(A2)

        return y_pred

    def optimize(self):
        # Initialize ants
        ants = []
        for i in range(self.num_ants):
            ant = {'weights': np.zeros(self.weights), 'pheromones': np.zeros_like(self.pheromones)}
            ants.append(ant)

        # Iterate over generations
        for gen in range(self.num_generations):
            # Generate ants
            for ant in ants:
                ant['weights'], ant['memory'] = self.generate_ant()
            
                    # Evaluate ants and update pheromones
            for ant in ants:
                y_pred = self.forward(self.X_train, ant['weights'])
                loss = log_loss(self.y_train, y_pred)
                ant['fitness'] = loss
                ant['pheromones'] += 1 / loss

            self.pheromones = self.update_pheromones(ants)

            # Traverse ants
            for ant in ants:
                ant = self.traverse(ant)

            # Print current generation's best ant
            best_ant = min(ants, key=lambda x: x['fitness'])
            print(f"Generation {gen + 1}: Best loss = {best_ant['fitness']}")
            #Acc:{accuracy_score(y_train,forward(inputs,best_ant['weights']))}")

        # Select the best ant from the final generation
        best_ant = min(ants, key=lambda x: x['fitness'])
        y_pred = self.forward(self.X_train, best_ant['weights'])
        accuracy = accuracy_score(self.y_train, y_pred)
        print(f"Best accuracy = {accuracy}")

        self.trained_weights = best_ant['weights']
    def predict(self,X_test):
        prediction=self.forward(X_test,self.trained_weights)
        return prediction
    
    def reports(self,X_test,y_test):
        prediction=self.forward(X_test,self.trained_weights)
        
        print(f"ACCURACY : {accuracy_score(y_test,prediction)}\n\n\
CLASSIFICATION REPORT :\n {classification_report(y_test,prediction)}")



In [13]:
aconn=AntColonyOptimizer(X_train,y_train)

In [14]:
aconn.optimize()

Generation 1: Best loss = 10.184209356039611
Generation 2: Best loss = 9.621027271834656
Generation 3: Best loss = 9.621027271834656
Generation 4: Best loss = 9.80875463323631
Generation 5: Best loss = 9.949550154287548
Generation 6: Best loss = 9.761822792885896
Generation 7: Best loss = 10.325004877090851
Generation 8: Best loss = 9.855686473586722
Generation 9: Best loss = 9.80875463323631
Generation 10: Best loss = 9.574095431484244
Generation 11: Best loss = 9.433299910433005
Generation 12: Best loss = 10.653527759543742
Generation 13: Best loss = 9.574095431484244
Generation 14: Best loss = 9.574095431484244
Generation 15: Best loss = 10.371936717441264
Generation 16: Best loss = 9.19864070868094
Generation 17: Best loss = 9.902618313937134
Generation 18: Best loss = 9.99648199463796
Generation 19: Best loss = 10.371936717441264
Generation 20: Best loss = 10.512732238492502
Generation 21: Best loss = 10.27807303674044
Generation 22: Best loss = 10.27807303674044
Generation 23: Be

In [15]:
aconn.reports(X_test,y_test)

ACCURACY : 0.7239583333333334

CLASSIFICATION REPORT :
               precision    recall  f1-score   support

           0       0.69      0.69      0.69        86
           1       0.75      0.75      0.75       106

    accuracy                           0.72       192
   macro avg       0.72      0.72      0.72       192
weighted avg       0.72      0.72      0.72       192

